<a href="https://colab.research.google.com/github/Noora-91/PortfolioProjects/blob/main/Sydney_airbnb_sql_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Summarize data of Airbnb Sydney!

<center>
<img src=https://live-production.wcms.abc-cdn.net.au/4d72b144a5661c8423472b84e0b11720 width="400"></center><br />

Welcome to my *SQL Crash Course* project! 


This is a project for CoRise SQL fundemental course that I submitted to recieve my certificate. In this project I have up with additionl questions than required at the end that I am able to solve by using SQL!
This project environment was configured by CoRise and I have made certain changes to it to fit my needs. All the SQL solutions here are attempted by me.





##### Prerequisite configuration
Below we install the software required to run this project. Please make sure to **RUN IT** by clicking on the play-button icon when you hover on the empty square bracket. And feel free to ignore the content of these two hidden cells.

IMPORTANT: These cells may have to be rerun every time you are away from the notebook a long time or access notebook on a different browser or a different laptop. 

In [ ]:
%%capture
!pip install git+https://github.com/sb2nov/sql-cc.git

import pandas as pd
from IPython.display import display, HTML
import sqlcc
from sqlcc import check

# Show all the rows (instead of only a few)
pd.set_option("display.max_rows", None)

# Set precision to max 2 decimals
pd.set_option('precision', 2)

# Set CSS Style for Table
# Make it work with night & light mode
# - Alternating rows
# - th elements
# - td elements
css_style = '''
<style>
  html {
    --td-font-color: black;
    --font-color: black;
    --background-color: #e0e0e0;
  }
  html[theme=dark] {
    --td-font-color: white;
    --font-color: black;
    --background-color: #6688ff;
  }
  th {
    background: #fbd44c;
    color: var(--font-color);
    font-size: 16px;
    text-align: center;
    font-weight: bold;
  }
  tr:nth-child(even) {
    background-color: var(--background-color);
    color: var(--font-color);
  }
  td {
    font-size: 14px;
    color: var(--td-font-color);
  }
</style>
'''


def run(sql_query):
  df = sqlcc.run(sql_query)
  
  # Puts the scrollbar next to the DataFrame
  display(HTML(css_style + 
               "<div style='max-height: 500px; overflow: auto; width: fit-content; border-style: solid;" + 
               " border-width: 1px; border-color: #0139fe; font-family: GT Planar,Inter,Arial,sans-serif;'>" +
               df.style.render() +
               "</div>"))

## Introduction

The picture below show the table that are accessible for information regarding the **listings**, **neighbourhoods**, **reviews**, and **calendar** of the Airbnb Sydney. Each of these have their own separate table, and follow the structure as shown.
<center>
<img src="https://corise.com/static/course/sql-crash-course/assets/cl9owz44x005e13a00fb5fhl0/Screen Shot 2022-10-25 at 5.42.23 PM.png" height="450">


### It's time to have a look through all the data sets available.

In [ ]:
query = """
SELECT * FROM neighbourhoods
"""
run(query)

,neighbourhood_id,neighbourhood
0,0,Ashfield
1,1,Auburn
2,2,Bankstown
3,3,Blacktown
4,4,Botany Bay
5,5,Burwood
6,6,Camden
7,7,Campbelltown
8,8,Canada Bay
9,9,Canterbury


In [ ]:
query = """
SELECT * FROM listings
"""
run(query)

,listing_id,listing,host_id,host,neighbourhood_id,room_type,price_in_dollar
0,574105250645758899,Peaceful 1 Bedroom Apartment in Bondi Beach,109067745,Andrew,35,Entire home/apt,150.00
1,7874902,Bondi Vibes - Funky Designer Studio,41506490,Alex White,35,Entire home/apt,99.00
2,4575789,Nice studio close to the beach!,22980172,María,35,Entire home/apt,125.00
3,23077495,Just bring your beach towel,1305312,Gladys,35,Entire home/apt,119.00
4,657377039990074158,Stylish lite 2b+2bth mod secure Beach apt with pkg,285488167,Rick,35,Entire home/apt,408.00
5,53798702,Spacious and clean 2-bedroom apartment in Bondi,244604436,Tiina,35,Entire home/apt,96.00
6,4344478,Bondi Beach Apartment 50m to beach,22553304,Stephen,35,Entire home/apt,101.00
7,48699778,Calm & Coastal: Bronte Beach Studio with Parking,185783910,Annie,35,Entire home/apt,159.00
8,12072720,Minutes to Bronte beach Ocean views,11745874,Angelika,35,Entire home/apt,210.00
9,20255786,Terrace in heart of Bondi Junction,18901875,Astrid And Nick,35,Entire home/apt,850.00


In [ ]:
query = """
SELECT * FROM calendar
"""
run(query)

,listing_id,date,available,price_in_dollar,minimum_nights
0,4344478,2022-09-09T00:00:00Z,0,101.00,90
1,4344478,2022-09-10T00:00:00Z,0,101.00,90
2,4344478,2022-09-11T00:00:00Z,0,101.00,90
3,4344478,2022-09-12T00:00:00Z,0,101.00,90
4,4344478,2022-09-13T00:00:00Z,0,101.00,90
5,4344478,2022-09-14T00:00:00Z,0,101.00,90
6,4344478,2022-09-15T00:00:00Z,0,101.00,90
7,7581665,2022-09-09T00:00:00Z,1,300.00,90
8,7581665,2022-09-10T00:00:00Z,0,300.00,90
9,7581665,2022-09-11T00:00:00Z,0,300.00,90


In [ ]:
query = """
SELECT * FROM reviews
"""
run(query)

,listing_id,review_id,date,reviewer_id,reviewer,comments
0,2033151,9717326,2014-01-06T00:00:00Z,10558887,James,"Great host, great location and very comfortable room. Many thanks."
1,4344478,62879688,2016-02-18T00:00:00Z,40858536,Andrew,"We had a fantastic time in Bondi and staying at Stephen's pad made it more worthwhile. Besides being located a few meters from the beach... gym, bus stops, eateries and supermarkets were all less than a 5min walk away. Stephen was a legend being just a text away if we needed any help. Top notch and friendly guy! We'll be back fosho!"
2,7846383,114619982,2016-11-19T00:00:00Z,73029331,Florian,It was very nice to stay two weeks by Marcela. The flat war nice and clean and near to the beach. I allways eat my breakfast on the balconary. Short bus trip to city. Marcela helps me when i have questions about activities in Sydney.Thank you for the nice time.
3,16411678,123595331,2016-12-29T00:00:00Z,64486440,Kristin,The host canceled this reservation 3 days before arrival. This is an automated posting.
4,16411678,140591929,2017-03-30T00:00:00Z,14665890,Shao Ing,"Alzbeta's studio is well look after, spacious and tastefully crafted. Clean yet cozy. Kitchen is well-equiped with nice setting. I enjoy the proximity to local amenities and various food choices. Alzbeta has been great with communication and a pleasure to meet in person. This is one great little studio that I would stay again. Many thanks!"
5,7874902,197790998,2017-09-27T00:00:00Z,7140613,Anh,Cosy little place in a great location. Close to Bondi beach and shops. Good amenities and the bed was really comfortable!!
6,20628052,199418949,2017-10-01T00:00:00Z,477117,Cristina,"Claire’s place was a wonderful place to base ourselves over a long weekend. The apartment is private with a great courtyard, which was really handy to dry towels and beach gear since it’s walking distance to the beach! The bed was comfortable and the kitchen was equipped with everything we needed. Claire was an incredibly hospitable host, helpful, and always quick to respond to any questions we had. We could not have found a better spot! Thank you so much Claire and Greta!"
7,22311178,226116570,2018-01-08T00:00:00Z,53414447,Lindsay & Mark,Kens place will hopefully make a great place to saty but its brand new and our experience had prblems No fridge it had been doscovered broke that morning this is ion the day after Sydney experienced 47 degress so we lost food and drink Cockroaches 4 in number throught the night and a big spider so needs pest conrtol and a tidy up of the boxes and builders rubble outsde tahts attaracting pests. Once thats done it will be fine
8,20628052,226117879,2018-01-08T00:00:00Z,59958313,Chris,"Perfect apartment for a couple. Great location only 5min walk to the beach as well as to the Manly ferry that takes you into central Sydney. Near to all the local bars, restaurants, coffee shops and supermarket but in a quiet location with a lovely garden. Hosts were fantastic with many nice touches including use of herbs and chillis from the garden as well as providing many essentials like milk and tea bags etc. Flexible and easy check in / out as well as lots of tips on where to go in Manly. We look forward to staying here again in the future!"
9,10549608,233186607,2018-02-08T00:00:00Z,84529327,Nick,The host canceled this reservation 40 days before arrival. This is an automated posting.


I was requested to write a SQL query that will give the following information about the host whose name starts with Jos... that we are not quite sure about within the neighbourhood_id 21. He had called earlier this morning complaining that his listing in the AirBnb never gets booked and is very dissapointed. Lets get to work to undestand why this could be happening!!!

The manual way of getting this could be to find 
*   the neighbourhood name
*   the availability of the room on the calendar
*   if there are any reviews written for the room

later we will see the aggregated way using joins to get the same answer.

Printing a list of hosts whose names start with the letters Jos...

In [ ]:
query = """
SELECT * 
FROM listings 
WHERE host LIKE 'Jos%';
"""

run(query)

,listing_id,listing,host_id,host,neighbourhood_id,room_type,price_in_dollar
0,22296011,Large private room on Camperdown park & Newtown,10873080,Joshua,21,Private room,40.00


Well! his name is Joshua and We identified the neighbourhood_id is "21" so now we can query on the neighbourhoods table

In [ ]:
query = """
SELECT * 
FROM neighbourhoods 
WHERE neighbourhood_id = 21;
"""
run(query)

,neighbourhood_id,neighbourhood
0,21,Marrickville


Lets try to get all the Availability details from the calender table for the listing_id "22296011"

In [ ]:
query = """
SELECT * 
FROM calendar 
WHERE listing_id = 22296011;
"""
run(query)

,listing_id,date,available,price_in_dollar,minimum_nights
0,22296011,2022-09-09T00:00:00Z,0,40.00,90
1,22296011,2022-09-10T00:00:00Z,0,40.00,90
2,22296011,2022-09-11T00:00:00Z,0,40.00,90
3,22296011,2022-09-12T00:00:00Z,0,40.00,90
4,22296011,2022-09-13T00:00:00Z,0,40.00,90
5,22296011,2022-09-14T00:00:00Z,0,40.00,90
6,22296011,2022-09-15T00:00:00Z,0,40.00,90


It seem there are no availibility for this specific listing and that should be the reason for this listing to not getting booked. By the way lets also investigate to see if there are any bad experience for this listing through analyzing the reviews table.

In [ ]:
query = """
SELECT * 
FROM reviews 
WHERE listing_id = 22296011;
"""

run(query)

,listing_id,review_id,date,reviewer_id,reviewer,comments


Ooooops! There isn't any single review or comment. We can solve the problem by fixing the availability in the calender. 

We can find that the host's name is "Joshua." He lives in Marrickville, has no availablility all week, and no reviews have been made for his room.

Let us find 5 listing with most reviews

In [ ]:
query = """
SELECT COUNT(listing_id), listing_id
FROM reviews
GROUP BY listing_id
ORDER BY COUNT(listing_id) DESC
LIMIT 5;
"""
run(query)

,COUNT(listing_id),listing_id
0,53,22311178
1,32,33757329
2,9,37999560
3,9,20628052
4,8,45917438


Let us also find to whom these listings belong and the neibourhood of these listings.

In [ ]:
query = """
SELECT reviews.listing_id, COUNT(reviews.listing_id), listings.host_id, listings.host
FROM reviews
LEFT JOIN listings
ON listings.listing_id = reviews.listing_id
GROUP BY reviews.listing_id
ORDER BY COUNT(reviews.listing_id) DESC
LIMIT 5;
"""
run(query)

,listing_id,COUNT(reviews.listing_id),host_id,host
0,22311178,53,163113803,Ken
1,33757329,32,254427140,The Jensen Potts Point
2,37999560,9,71234146,Derek
3,20628052,9,3444681,Claire
4,45917438,8,215158027,Peter & Stephanie


Now let us find the 10 least reviewed listed_ids and to whom these listings belong and the neibourhood of these listings.

In [ ]:
query = """
SELECT reviews.listing_id, COUNT(reviews.listing_id), listings.host_id, listings.host
FROM reviews
LEFT JOIN listings
ON listings.listing_id = reviews.listing_id
GROUP BY reviews.listing_id
ORDER BY COUNT(reviews.listing_id) ASC
LIMIT 10;
"""
run(query)

,listing_id,COUNT(reviews.listing_id),host_id,host
0,7846383,1,40070341,Marcela
1,7874902,1,41506490,Alex White
2,9376988,1,11196275,Emma
3,10549608,1,23953750,Cyrus
4,50118577,1,169795871,Yihan
5,52704433,1,54924558,Hudson
6,54362721,1,51257703,Barry
7,697876946371921032,1,457666854,Zeam
8,2033151,2,5727462,Julia
9,34027782,2,163419570,Zach


Different rooms types available for booking

In [ ]:
query = """
SELECT DISTINCT room_type
FROM listings;
"""
run(query)

,room_type
0,Entire home/apt
1,Private room
2,Hotel room


Available units for eacg category

In [ ]:
query = """
SELECT room_type, COUNT(room_type)
FROM listings
GROUP BY room_type;
"""
run(query)

,room_type,COUNT(room_type)
0,Entire home/apt,46
1,Hotel room,1
2,Private room,16


Out of the three different room_type which type has acquired more reviews

In [ ]:
query = """
SELECT 
  listings.room_type, 
  COUNT(listings.room_type) AS No_of_room_type, 
  COUNT(reviews.listing_id) AS No_of_reviews
FROM listings
LEFT JOIN reviews
  ON listings.listing_id = reviews.listing_id
GROUP BY room_type;
"""
run(query)

,room_type,No_of_room_type,No_of_reviews
0,Entire home/apt,150,122
1,Hotel room,32,32
2,Private room,20,8


In [ ]:
query = """
SELECT  
  listings.listing_id,
  listings.room_type,
  COUNT(reviews.listing_id) AS No_of_reviews
FROM listings
LEFT JOIN reviews
  ON listings.listing_id = reviews.listing_id
GROUP BY listings.listing_id
ORDER BY COUNT(reviews.listing_id) DESC;
"""
run(query)

,listing_id,room_type,No_of_reviews
0,22311178,Entire home/apt,53
1,33757329,Hotel room,32
2,37999560,Entire home/apt,9
3,20628052,Entire home/apt,9
4,45917438,Entire home/apt,8
5,44284905,Entire home/apt,7
6,41983356,Entire home/apt,6
7,4344478,Entire home/apt,6
8,574105250645758899,Entire home/apt,5
9,48699778,Entire home/apt,4


Let's try to print listing_ids that do not have any reviews to them

In [ ]:
query = """
SELECT  
  listings.listing_id,
  listings.room_type,
  COUNT(reviews.listing_id) AS No_of_reviews
FROM listings
LEFT JOIN reviews
  ON listings.listing_id = reviews.listing_id
WHERE reviews.listing_id IS NULL
GROUP BY listings.listing_id;
"""
run(query)

,listing_id,room_type,No_of_reviews
0,4575789,Entire home/apt,0
1,4632356,Entire home/apt,0
2,7581665,Entire home/apt,0
3,8476652,Private room,0
4,8925052,Entire home/apt,0
5,9012529,Entire home/apt,0
6,9354832,Entire home/apt,0
7,10103846,Entire home/apt,0
8,10111788,Private room,0
9,10729688,Entire home/apt,0


Let's have a look at the 10 most expensive listings

In [ ]:
query = """
SELECT calendar.listing_id, listings.room_type, calendar.price_in_dollar
FROM calendar
LEFT JOIN listings
  ON calendar.listing_id = listings.listing_id
ORDER BY calendar.price_in_dollar DESC
LIMIT 10;
"""
run(query)

,listing_id,room_type,price_in_dollar
0,10111788,Private room,99999.00
1,10111788,Private room,99999.00
2,9376988,Entire home/apt,2000.00
3,9376988,Entire home/apt,2000.00
4,9376988,Entire home/apt,2000.00
5,9376988,Entire home/apt,2000.00
6,9376988,Entire home/apt,2000.00
7,9376988,Entire home/apt,2000.00
8,9376988,Entire home/apt,2000.00
9,46159786,Entire home/apt,1000.00


Number of listings in each neighbourhood

In [ ]:
query = """
SELECT 
  neighbourhoods.neighbourhood_id,
  neighbourhoods.neighbourhood,
  COUNT(listings.listing_id) AS no_of_listings
FROM listings
LEFT JOIN neighbourhoods
  ON neighbourhoods.neighbourhood_id = listings.neighbourhood_id
GROUP BY neighbourhoods.neighbourhood_id
ORDER BY no_of_listings DESC;
"""
run(query)

,neighbourhood_id,neighbourhood,no_of_listings
0,35,Waverley,13
1,32,Sydney,10
2,37,Woollahra,5
3,20,Manly,5
4,27,Randwick,4
5,29,Ryde,3
6,26,Pittwater,3
7,23,North Sydney,3
8,31,Sutherland Shire,2
9,21,Marrickville,2


Find the cheapest booking in the neighbourhood "Waverley" with the most reviews

In [ ]:
query = """
SELECT 
  listings.listing_id, 
  listings.price_in_dollar,
  neighbourhoods.neighbourhood
FROM listings
LEFT JOIN neighbourhoods
  ON neighbourhoods.neighbourhood_id = listings.neighbourhood_id
WHERE neighbourhoods.neighbourhood = 'Waverley' AND
  listings.price_in_dollar > 800
GROUP BY neighbourhoods.neighbourhood_id;
"""
run(query)

,listing_id,price_in_dollar,neighbourhood
0,20255786,850.00,Waverley


In [ ]:
query = """
SELECT 
  listings.price_in_dollar AS list_price,
  Calendar.price_in_dollar
FROM listings
LEFT JOIN calendar
  ON listings.price_in_dollar = Calendar.price_in_dollar;
"""
run(query)

,list_price,price_in_dollar
0,150.00,150.00
1,150.00,150.00
2,150.00,150.00
3,150.00,150.00
4,150.00,150.00
5,150.00,150.00
6,150.00,150.00
7,150.00,150.00
8,150.00,150.00
9,150.00,150.00
